In [1]:
### ignore warning 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import numpy as np
import pandas as pd
import re
import glob
import spacy 
import nltk
import os
import nltk
from nltk.tokenize import TreebankWordTokenizer ,word_tokenize
from nltk.stem import PorterStemmer , WordNetLemmatizer
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams

# nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk import everygrams

# display  max row 
#pd.set_option("display.max_rows", None)

## tqdm 
from tqdm._tqdm_notebook import tqdm_notebook,tqdm
tqdm_notebook.pandas()

from spacy import displacy
nlp1 = spacy.load('en_core_web_sm') 

[nltk_data] Downloading package punkt to /home/nick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_new_test = pd.read_csv('./new_test_set_added.csv')
df_new_test.shape

(127, 25)

In [3]:
df_new_test.reset_index(inplace=True)

In [31]:
def prep_data(data) :
    word_domain = []
    wpt = nltk.WordPunctTokenizer()
    stop_word  = stopwords.words('english')
    #data = data_test
    data = data.strip()
    # optional  base on inspection  
    #data  =  data.lower()
    ########################### 
    result = string.punctuation
    ### Clean HTML Tacontents = df_clean.content.to_list()
    data = re.sub(r'\n','',data)
    data = re.sub(r'\xa0','',data)
    data = re.sub(r"\\","",data)
    data = re.sub(r'<li>','',data)
    data = re.sub(r'</li>','',data)
    data = re.sub(r'</ul>','',data)
    data = re.sub(r'<td>','',data)
    data = re.sub(r'</td>','',data)
    
    
    data= re.sub(r'\u200b', '', data)
    data = re.sub(r'[ๆฯ!#$&%\"\'()*+,-./:;<=>?@\[\]\\^_`{}|~]',' ', data)
    data = re.sub(r'\d',' ', data)
    data = re.sub(r'\n', ' ', data)
    #data = re.sub(r'[^a-zA-Z\s]', '', data, re.I|re.A)
    data = re.sub(r'・','',data)
    

   
    lst_pun = [i for i in result] + ['•'] +['“','”','’']+['©']+['£']+['—'] +['-']
    data = data.replace('|',' ')
    data = data.replace('-', ' ')
    data = data.replace('|****|',' ')
    data = wpt.tokenize(data)
    data =  ' '.join( i for i in data if i not in lst_pun and i.isnumeric() == False ) 
    data  =  nltk.tokenize.word_tokenize(data)
    
    data =  [i for i in data if i not in  stop_word ]
    data   = ' '.join(data)
    return data



def ner_process(sentens_string) :
    doc1 = nlp1(sentens_string) 
    sentenss = []
    for sr in  doc1 :
        if sr.ent_iob_  == 'O' :
               sentenss.append(str(sr)) 
    return ' '.join(sentenss)  
    #data  = list(everygrams(data, 1, 3))
    
def noun_only(x):
    x  = word_tokenize(x)
    pos_comment = nltk.pos_tag(x)
    filtered = [word[0] for word in pos_comment if word[1] in ['NN']]
    # to filter both noun and verbs
    #filtered = [word[0] for word in pos_comment if word[1] in ['NN','VB', 'VBD', 'VBG', 'VBN', 'VBZ']]
    return filtered


from sklearn.feature_extraction.text import CountVectorizer
def count_vector(key_word,content) :
    key_word = [' '.join(key_word)]
    print(len(key_word))
    corpus =  key_word + content
    #corpus =  econ_all2 + xx_list
    #corpus_str = [' '.join(corpus)]
    #print(len(corpus)
    cvec = CountVectorizer(analyzer = lambda x:x.split(' ')) #
    cvec.fit_transform(corpus)
    print(len(cvec.vocabulary_))

    train_bow = cvec.transform(corpus)
    print(len(train_bow.toarray()))
    return train_bow.toarray() , cvec.get_feature_names() #train_bow.toarray() 
    
## LDA GLOVE CORPUS


def knn(W, x, k):
    # The added 1e-9 is for numerical stability
    cos = nd.dot(W, x.reshape((-1,))) / (
        (nd.sum(W * W, axis=1) + 1e-9).sqrt() * nd.sum(x * x).sqrt())
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec,
                    embed.get_vecs_by_tokens([query_token]), k+1)
    list_sym = [] 
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        #print('cosine sim=%.3f: %s' % (c, (embed.idx_to_token[i])))
        list_sym.append(embed.idx_to_token[i])
        
    return   list_sym

df_new_test['clean_content_ner'] =  df_new_test.progress_apply(lambda x: ner_process(x['Title'] + ' ' + x['Content']), axis=1)
df_new_test['clean_content_prep'] = df_new_test['clean_content_ner'].progress_apply(prep_data) 
df_new_test['clean_content_prep_last'] =  df_new_test['clean_content_prep'].progress_apply(lambda x : word_tokenize(x))
df_new_test['clean_content_prep_pos_n'] =  df_new_test['clean_content_prep'].progress_apply(noun_only)

  0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

## Read File Corpus

In [39]:
path = '/home/nick/DITP/trade-midas/ditp3_run_test/'

#econ 
econ_dict =  pd.read_csv(f'{path}/econ_dict_corpus.csv', index_col=False)
econ_dict.Key_word = econ_dict.Key_word.progress_apply(prep_data)
econ_dict.Relate_word = econ_dict.Relate_word.progress_apply(prep_data)


#related product 
relate_product = pd.read_csv(f'{path}/related_product_corpus.csv')
relate_key = relate_product['keyword'].to_list()
relate_product.keyword = relate_product.keyword.progress_apply(prep_data)


rp =  relate_product.keyword.to_list()
#all_p = all_product.keyword.to_list()
#location_l = location.keyword.to_list()
econ_d_key  =  econ_dict.Key_word.to_list()
econ_d_related  =  econ_dict.Relate_word.to_list()


econ_all = econ_d_key + econ_d_related
econ_all =  list(set(econ_all))
econ_all = list(map( lambda x : x.lower(), econ_all))




! pip install more-itertools
from functools import reduce
import more_itertools ## reduct list nest to sigle list 
rp = list(map( lambda x : x.lower() , rp))



rp =  list(filter(lambda x : len(x) > 2 , rp))
print('Before' , len(rp))
rp_new =  [ i.split(' ')  if len(i.split(' ')) > 1  else i  for i in rp ]
rp_new = list(more_itertools.collapse(rp_new))
rp_new =  list(set(rp_new))
rp_new =  list(filter( lambda x : len(x) > 3,rp_new )) + ['oil'] 
print('Final',len(rp_new))


except_list_rp = ['mens','woman','swim', 'basic','semi','fats','long','sale','extra','room','shorts','mixed','queen'
                 ,'hair','form','base','cuts','cars','type','root','prcs','swim','thai','women','japan','acid','beds'
                 ,'sound','tree','clad','parts','super','clip'] +  ['wind','block','sheets','star','dried'
                       ,'roots','example','shock','slips','cubeb','white','brown'
              ,'broken','coin','aircraft','preparations','long','ephedras','room','short','special','mixed','overalls'
              ,'pipes','ensembles' ,'capacity','aloe','japan' ,'flax','women','indonesian' ,'elemi','leaves','plugs'
              ,'root','gear','columns' , 'swim' ,'type' ,'boys','lead','sassafras','semi','basic','bran','drms','load','goods'
               ,'prcs','spokes','kind','shower','sheet','broke','purpose','base','road','shorts','super','tools','toilet','part'
               ,'logs','thai','tons','chinese','care','photographs','latex','vera','holy','form','sound','sage','bird','forms',
               'wall','hair','logs'

              ]

rp_new_clean =  [ i  for i in rp_new if i not in   except_list_rp ]

rp_new_clean = list(set(rp_new_clean))


#### Econ dictionary 

from functools import reduce
import more_itertools ## reduct list nest to sigle list 
econ_all1 =  [  iu.split(' ') if len(iu.split()) > 1 else iu for iu in  econ_all  ]
econ_all1 = list(more_itertools.collapse(econ_all1))
econ_all1 =  list(set(econ_all1))



except_word  = ['say','zero','numbers','terms','mac','boom','ppp','account','asian','path','simple','rate','null'
    ,'mode','long','zone','exit','repo','area','man','areas','common','mean','mixed','spot','tick','hot'
,'base','number','best','entry','bust','plaza','point','non','okun','neo','hard','tiger','way','the','paris','big','plan',
'third','normal','fine','job','and','command','soft','aid','work','game','ngo','search','last','nairu','full','new','homo'
,'alan','sum','euro','john','club','government','bills','out','good','beta','lbo','open','all','random','right','for'
 ,'closed','current','security','environmental','']
econ_all2 =  [ i   for i in econ_all1 if len(i) > 2 and i not in except_word  ]

all_word  =  econ_all2 
new_all_word =   list(filter(lambda x :  len(x) >  2 , all_word ))
new_all_word =  [ i.lower() for i in new_all_word ]



## related word LDA 
df_lda =  pd.read_csv('./lda_keyword_phase2.csv')
de_set = df_lda['keyword_lda'].to_list()


  0%|          | 0/3147 [00:00<?, ?it/s]

  0%|          | 0/3147 [00:00<?, ?it/s]

  0%|          | 0/967 [00:00<?, ?it/s]

Before 963
Final 658


In [121]:

re_product_list = []
for io in   tqdm(df_new_test.to_dict('record')):
    dic_cont = [ i.lower() for i in  io['clean_content_prep_pos_n']] #
    
    #print(dic_cont)
#     print(rp_new_clean)
    xxee = list(filter( lambda r :  r in  rp_new_clean , dic_cont))
    #print(xxee)
    re_product_list.append(' '.join(xxee))
    
    
    
econ_list = []
for ii in   tqdm(df_new_test.to_dict('record')):
    dic_cont =  ii['clean_content_prep_last']               
    xxdd = list(filter( lambda x :  x in new_all_word , dic_cont)) 
    econ_list.append(' '.join(xxdd))
    
    
lda_list = []
for ii in   tqdm(df_new_test.to_dict('record')):
    dic_cont =  ii['clean_content_prep_last']               
    xxlda = list(filter( lambda x :  x in de_set , dic_cont)) 
    lda_list.append(' '.join(xxlda))    
    

  0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

In [131]:
#Economy dictionary

econ_array,econ_feature = count_vector(new_all_word,econ_list)
df_econ =pd.DataFrame( econ_array , columns = econ_feature, index = range(len(econ_list)+1))
try:
    df_econ.drop(columns  = [''] , inplace=True)
except: 
    pass
df_econ['sum_unique'] = df_econ.replace(0, np.nan).notnull().sum(axis= 1)#.notnull().count(axis=1)
df_econ['sum_total'] =  df_econ.iloc[:, : -1 ].sum(axis= 1)

df_econ.drop(index= 0 , inplace=True)
df_econ_index  =  df_econ.index.to_list()
df_econ_index =  [i-1 for i in df_econ_index ]
df_econ.index = df_econ_index

q3 = 7
df_econ1 = df_econ[df_econ['sum_unique'] >= q3]

# related propuct  
product_array,product_feature = count_vector(rp_new_clean,re_product_list)
df_product_related =pd.DataFrame( product_array , columns = product_feature, index = range(len(re_product_list)+1))
try:
    df_product_related.drop(columns = [''], inplace=True)
except: 
    pass
df_product_related['sum_unique'] = df_product_related.replace(0, np.nan).notnull().sum(axis= 1)#.notnull().count(axis=1)
df_product_related['sum'] =  df_product_related.iloc[:, : -1 ].sum(axis= 1)


df_product_related.drop(index=0 , inplace=True)
df_product_related_index  =  df_product_related.index.to_list()
df_product_related_index =  [i-1 for i in df_product_related_index ]
df_product_related.index = df_product_related_index

q3_related = 2
df_product_related1 = df_product_related[df_product_related['sum'] >= q3_related]

## LDA key word
lda_array,product_feature_lda = count_vector(de_set,lda_list)
df_lda_related =pd.DataFrame( lda_array , columns = product_feature_lda, index = range(len(lda_list)+1))
df_lda_related.drop(columns = [''], inplace=True,errors='ignore')
df_lda_related['sum_unique'] = df_lda_related.replace(0, np.nan).notnull().sum(axis= 1)#.notnull().count(axis=1)
df_lda_related['sum_total'] =  df_lda_related.iloc[:, : -1 ].sum(axis= 1)

df_lda_related.drop( index=0, inplace=True )
df_lad_related_index  =  df_lda_related.index.to_list()
df_lad_related_index =  [i-1 for i in df_lad_related_index ]
df_lda_related.index = df_lad_related_index

q3_lda =  20
df_lda_related1 = df_lda_related[df_lda_related['sum_unique'] >=q3_lda]

1
650
128
1
569
128
1
881
128


In [132]:
df_econ['sum_unique'][15]


6

In [133]:
i = 15
df_econ['sum_unique'][i],df_product_related['sum_unique'][i],df_lda_related['sum_unique'][i]

(6, 2, 12)

In [134]:
# LDA realted corpus
# from mxnet import nd
# from mxnet.contrib import text
# glove_6b50d = text.embedding.create(
#     'glove', pretrained_file_name='glove.840B.300d.txt')

all1 = df_econ1[['sum_unique','sum_total']].merge(df_product_related1[['sum_unique','sum']], right_index=True, left_index=True  )
all2 = all1.merge(df_lda_related1[['sum_unique','sum_total']],  right_index=True, left_index=True )
#df_product_related
x2 = df_new_test[['Content','Verified by\nDITP']].merge(all2, how = 'left',right_index=True, left_index=True )

In [135]:
x2[(x2['Verified by\nDITP'] == 'yes') & (x2['sum_unique_x'].isna())  ].shape #

(27, 8)

In [73]:
x1[(x1['Verified by\nDITP'] == 'yes') & (x1['sum_unique'] ==0 ) ].shape

# 101  yes yes 
# 1     yes no 

# no  yes  14  high gain  
# no no    6 


NameError: name 'x1' is not defined

In [70]:
x2 = df_new_test[['Content','Verified by\nDITP']].merge(df_econ[['sum_total','sum_unique']], how = 'left',right_index=True, left_index=True )

In [88]:

x2.replace(np.nan ,  0, inplace=True)
x2[(x2['Verified by\nDITP'] == 'no') & (x2['sum_unique'] !=0 ) ].shape
# total  yes =  102 
# yes yes =  63
# yes  no = 39 
# no no   2
# no yes   18 




(18, 4)

In [48]:
# intersaction  with  3 domain 
all1 = df_econ0[['sum_unique','sum_total']].merge(df_product_related0[['sum_unique','sum']],
                                                  right_index=True, left_index=True  )
all2 = all1.merge(df_lda_related0[['sum_unique','sum_total']],  right_index=True, left_index=True )

NameError: name 'df_econ0' is not defined

In [47]:
all2.rename(columns={'sum_unique_x':'sum_unique_econ',
                     'sum_total_x':'sum_total_econ',
                     'sum_unique_y' : 'sum_unique_relate_pro',
                     'sum' : 'sum_total_relate_pro',
                     'sum_unique' : 'sum_unique_lda',
                     'sum_total_y' : 'sum_total_lda'
                    }, inplace=True)

NameError: name 'all2' is not defined